<a href="https://colab.research.google.com/github/Roubish/Deep-Learing-Project-/blob/main/vehicle_detection%20with_custom_data_yolov4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
RESEARCH_DIR = "/content/drive/MyDrive/Roubish/vehicle_detection"
os.chdir(RESEARCH_DIR)
os.getcwd()

In [ ]:
%cd /content/drive/MyDrive/Roubish/vehicle_detection/yolov4

In [ ]:
# #mount drive
# %cd ..
# from google.colab import drive
# drive.mount('/content/gdrive')

# # this creates a symbolic link so that now the path /content/gdrive/My\ Drive/ is equal to /mydrive
# !ln -s /content/gdrive/My\ Drive/ /mydrive

# # list the contents of /mydrive
# !ls /mydrive

# #Navigate to /mydrive/yolov4
# %cd /mydrive/yolov4

In [ ]:
!git clone https://github.com/AlexeyAB/darknet

In [ ]:
# change makefile to have GPU and OPENCV enabled
# also set CUDNN, CUDNN_HALF and LIBSO to 1

%cd darknet/
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile
!sed -i 's/LIBSO=0/LIBSO=1/' Makefile

In [ ]:
# build darknet 
!make

In [ ]:
# Clean the data and cfg folders first except the labels folder in data which is required

%cd data/
!find -maxdepth 1 -type f -exec rm -rf {} \;
%cd ..

%rm -rf cfg/
%mkdir cfg

In [ ]:
# Unzip the obj.zip dataset and its contents so that they are now in /darknet/data/ folder 

!unzip /content/drive/MyDrive/Roubish/vehicle_detection/yolov4/obj.zip -d data/

In [ ]:
# Copy the yolov4-custom.cfg file so that it is now in /darknet/cfg/ folder 

!cp /content/drive/MyDrive/Roubish/vehicle_detection/yolov4/yolov4-custom.cfg cfg

# verify if your custom file is in cfg folder
!ls cfg/

In [ ]:
# Copy the obj.names and obj.data files from your drive so that they are now in /darknet/data/ folder 

!cp /content/drive/MyDrive/Roubish/vehicle_detection/yolov4/obj.names data
!cp /content/drive/MyDrive/Roubish/vehicle_detection/yolov4/obj.data  data

# verify if the above files are in data folder
!ls data/

In [ ]:
# Copy the process.py file to the current darknet directory 

!cp /content/drive/MyDrive/Roubish/vehicle_detection/yolov4/process.py .

In [ ]:
# run process.py ( this creates the train.txt and test.txt files in our darknet/data folder )
!python process.py

# list the contents of data folder to check if the train.txt and test.txt files have been created 
!ls data/

In [ ]:
# Download the yolov4 pre-trained weights file
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.conv.137

In [ ]:
# train your custom detector! (uncomment %%capture below if you run into memory issues or your Colab is crashing)
# %%capture

!./darknet detector train /content/drive/MyDrive/Roubish/vehicle_detection/yolov4/darknet/data/obj.data /content/drive/MyDrive/Roubish/vehicle_detection/yolov4/darknet/cfg/yolov4-custom.cfg yolov4.conv.137 -dont_show -map

In [ ]:
# define helper function imShow
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

In [ ]:
#run detector on images captured by webcam for your custom YOLOv4 trained model
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

from IPython.display import Image
try:
  filename = take_photo()
  print('Saved to {}'.format(filename))
  
  # Show the image which was just taken.
  display(Image(filename))
except Exception as err:
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
  print(str(err))



In [ ]:
!./darknet detector test data/obj.data cfg/yolov4-custom.cfg /content/drive/MyDrive/Roubish/vehicle_detection/yolov4/training/yolov4-custom_best.weights /content/drive/MyDrive/Roubish/vehicle_detection/yolov4/darknet/image24.jpg -thresh 0.5
print('image24.jpg') 

In [ ]:
# run your custom detector on a video with this command (upload a video to your google drive to test, the thresh flag sets the minimum accuracy required for object detection).This saves the output video with the detections in your output path

!./darknet detector demo data/obj.data cfg/yolov4-custom.cfg /content/drive/MyDrive/Roubish/vehicle_detection/yolov4/training/yolov4-custom_best.weights -dont_show /content/drive/MyDrive/Roubish/vehicle_detection/yolov4/darknet/vehical_test/vehical_videos.mp4 -i 0 -out_filename /content/drive/MyDrive/Roubish/vehicle_detection/yolov4/darknet/vehical_test/finalresult1.avi 